SQL:


In [16]:
%load_ext sql
# Connect to SQLite database (creates file if it doesn't exist)
%sql sqlite:///../database/Clinic.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Identify Purpose of the System:
1. Why is this db needed?
2. What real-world process we want to digitize?

Therefore, a clinical system will need to:
1. Store Doctor, Patient information
2. Allow Patient appointments
3. Store patients and treatment

Data to store:
Running a Clinic will require data like Patients, Doctors, Appointments, Treatment

Identifying Entities:
Entity identification is crucial for database creation. Entities allow us to decide on what tables we need for our database. It is important to focus on picking entities that are independent, logically separable, and posses their own attributes.

Creating tables:
We have to be cautious while creating tables in order to avoid outcomes like storing too much in one table, creating many fragmented tables, queries becoming unnecessarily complex.
For a clinic, we need tables such as Patients, Doctors, Appointments, Treatment

In [17]:
%%sql

DROP TABLE IF EXISTS Treatments;
DROP TABLE IF EXISTS Appointments;
DROP TABLE IF EXISTS Patients;
DROP TABLE IF EXISTS Doctors;

CREATE TABLE Doctors (
    doctor_id INTEGER PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    specialty VARCHAR(50) NOT NULL,
    hourly_rate DECIMAL(8, 2)
);

CREATE TABLE Patients (
    patient_id INTEGER PRIMARY KEY,
    name VARCHAR(50) NOT NULL,
    phone VARCHAR(15) UNIQUE
);

CREATE TABLE Appointments (
    appoint_id INTEGER PRIMARY KEY,
    patient_id INT NOT NULL,
    doctor_id INT NOT NULL,
    appoint_date DATE NOT NULL,
    status TEXT CHECK(status IN ('Scheduled', 'Completed', 'Cancelled')),
    FOREIGN KEY (patient_id) REFERENCES Patients(patient_id),
    FOREIGN KEY (doctor_id) REFERENCES Doctors(doctor_id)
);

CREATE TABLE Treatments (
    treatment_id INTEGER PRIMARY KEY,
    appoint_id INT NOT NULL,
    service_name VARCHAR(50) NOT NULL,
    cost DECIMAL(8, 2) NOT NULL,
    FOREIGN KEY (appoint_id) REFERENCES Appointments(appoint_id)
);

 * sqlite:///../database/Clinic.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

Identifying attributes:
Patients:
    patientid,
    name,
    name,
    phone

Appointments:
    appoint_id,
    patient_id,
    doctor_id,
    appointdate,
    status
    

In [18]:
%%sql

INSERT INTO Doctors (first_name, specialty, hourly_rate) VALUES
('Dr. Rima', 'Cardiology', 2500.00),
('Dr. Hari', 'General Practice', 1500.00);

INSERT INTO Patients (name, phone) VALUES
('Anil Shrestha', '9848000001'),
('Bimala Thapa', '9848000002'),
('Chetan Rai', '9848000003'); 

INSERT INTO Appointments (patient_id, doctor_id, appoint_date, status) VALUES
(1, 1, '2025-12-01', 'Completed'), 
(2, 2, '2025-12-01', 'Completed'), 
(1, 2, '2025-12-02', 'Scheduled'), 
(2, 1, '2025-12-03', 'Cancelled'); 

INSERT INTO Treatments (appoint_id, service_name, cost) VALUES
(1, 'ECG Scan', 1500.00), 
(1, 'Consultation', 2500.00), 
(2, 'Checkup', 1500.00); 

 * sqlite:///../database/Clinic.db
2 rows affected.
3 rows affected.
4 rows affected.
3 rows affected.


[]

Identifying relationships between entities
Patients -> Doctors     (Many-to-Many: M:N)
    Add a junction table that has both M and N as Foreign Key
Patients -> Appointments (One-to-Many: 1:N)
    Add Foreign Key to Many side
Doctors -> Appointments (One-to-Many: 1:N)
Appointments -> Treatments (One-to-Many: 1:N)

In [19]:
%%sql

SELECT service_name, cost
FROM Treatments
ORDER BY cost DESC
LIMIT 1;

 * sqlite:///../database/Clinic.db
Done.


service_name,cost
Consultation,2500


In [20]:
%%sql

SELECT appoint_id, appoint_date, status
FROM Appointments
ORDER BY appoint_date DESC
LIMIT 1;

 * sqlite:///../database/Clinic.db
Done.


appoint_id,appoint_date,status
4,2025-12-03,Cancelled


Counting Appointments by Doctor:
Aggregation and Grouping

Joins:
A join is combination of two or more tables based on a related column between them. A join allows us to query from two or more tables at once when they have certain relationship.

There are six types of joins. Let's start with the basic one - inner join.
1. Inner Join: It is used when we want to get only the rows that match among the joined tables. The tables are joined based on a matching condition.
Here, we want to find the doctors according to their specialty and the total number of appointments they have.

In [21]:
%%sql

SELECT
d.first_name AS Doctor_Name,
d.specialty,
COUNT(a.appoint_id) AS Total_Appointments
FROM Doctors d
INNER JOIN Appointments a ON d.doctor_id = a.doctor_id
GROUP BY d.first_name, d.specialty
ORDER BY Total_Appointments DESC;

 * sqlite:///../database/Clinic.db
Done.


Doctor_Name,specialty,Total_Appointments
Dr. Hari,General Practice,2
Dr. Rima,Cardiology,2


2. Left Join: Left join is used when we want to return all the rows from the "left" table and the matching rows from the "right" table. NULL is included when there are no values to return.

In [24]:
%%sql

SELECT a.appoint_id, d.first_name as doctor, d.specialty, p.name as patient_name, a.status 
FROM Doctors d
LEFT JOIN Appointments a on d.doctor_id=a.doctor_id
LEFT JOIN Patients p on p.patient_id=a.patient_id
ORDER BY d.first_name, a.appoint_date desc;

 * sqlite:///../database/Clinic.db
Done.


appoint_id,doctor,specialty,patient_name,status
3,Dr. Hari,General Practice,Anil Shrestha,Scheduled
2,Dr. Hari,General Practice,Bimala Thapa,Completed
4,Dr. Rima,Cardiology,Bimala Thapa,Cancelled
1,Dr. Rima,Cardiology,Anil Shrestha,Completed
